In [61]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re

In [33]:
result = requests.get("https://yugipedia.com/wiki/2022_Tin_of_the_Pharaoh%27s_Gods")
src = result.content
soup = BeautifulSoup(src, 'html.parser')

In [36]:
Jeff = soup.find('h1')

In [37]:
Jeff.text

"2022 Tin of the Pharaoh's Gods"

In [119]:
#Get the set page from https://yugipedia.com/wiki/Yugipedia
class TCG_set:
    def __init__(self, url):
        #Gives us url, set content df, number of cards, set name.
        
        #Getting the url we put in
        self.url = url
        result = requests.get(url)
        src = result.content
        soup = BeautifulSoup(src, 'html.parser')
        
        #Getting the name of the set
        set_name = soup.find('h1')
        self.set_name = set_name.text
        
        #Getting the content of the set
        card_info_headers = soup.find(attrs = {'class':'set-list'})

        #getting the name of each table column - not convinced we need this
        #col_name = []
        #headers = card_info_headers.find_all('th')
        #for header in headers:
        #    col_name.append(header.text)

        #split table data into the five categories from the title above
        card_number = []
        card_name = []
        card_rarity = []
        card_category = []
        card_print = []
        table_data = card_info_headers.find_all('td')
        for i in range(0,len(table_data)):
            if (i % 5 == 0):
                card_number.append(table_data[i].text)
            elif (i % 5 == 1):
                
                #get rid of the quote marks around the name
                
                name = table_data[i].text[1:len(table_data[i].text)-1]
                name = re.sub('/', '', name)
                card_name.append(name)
                
            elif (i % 5 == 2):
                card_rarity.append(table_data[i].text)
            elif (i % 5 == 3):
                card_category.append(table_data[i].text)
            else:
                card_print.append(table_data[i].text)

        set_content = pd.DataFrame({"Set Number" : card_number, "Name": card_name, "Rarity": card_rarity, "Category": card_category, "Print" : card_print})
        self.set_content = set_content
        
        #Getting the number of cards
        self.set_card_number = len(set_content)
        
        #Clean the name to put into a cardmarket url.
        re_set_name = self.set_name.replace(' ', '-')
        re_set_name = re.sub('\'','', re_set_name)

        #Number of index pages
        no_of_pages = (self.set_card_number // 20) + 1
        #Getting cardmarket index
        links = []
        for i in range(1, no_of_pages+1):
            cardmarket_result = requests.get('https://www.cardmarket.com/en/YuGiOh/Products/Singles/{}?sortBy=collectorsnumber_asc&site='.format(re_set_name) + str(i))
            src = cardmarket_result.content
            soup = BeautifulSoup(src, 'html.parser')
            card_price_info = soup.find_all(attrs = {'class':'col'})
            for entry in card_price_info:
                    if entry.find('a') != None:
                        cardmarket_url = entry.find('a')
                        if (cardmarket_url.attrs['href'] != 'https://www.facebook.com/CardmarketYugioh') and (cardmarket_url.attrs['href'] != '/en/YuGiOh/Policies/GeneralTermsAndConditions'):
                            links.append('https://www.cardmarket.com/' + cardmarket_url.attrs['href'])
        
        self.set_content['cardmarket url'] = links
        
        #link to seller location = UK
        uk_links = []
        for link in links:
            link = link + '?sellerCountry=13'
            uk_links.append(link)
        
        self.set_content['cardmarket url uk'] = uk_links 
        
    #Quick function to get the set content printed as a .csv file
    def content_to_csv(self):
        self.set_content.to_csv('{}.csv'.format(self.set_name))
        
    #This function gets us the price data for the last two weeks for each card in the set
    def cardmarket_price_last_two_weeks(self):
        df = MP22.set_content
        latest_date = []
        latest_price = []
        for j in range(111 ,len(list(df['cardmarket url uk']))):

            #Parsing the site for the data from the chart (last two weeks of avg sale price in Euros)
            url = list(df['cardmarket url uk'])[j]
            result = requests.get(url)
            src = result.content
            soup = BeautifulSoup(src, 'html.parser')
            card_price_info = soup.find_all(attrs = {'class':'chart-init-script'})

            #Getting the info we want from the parsing

            info = str(card_price_info[0])
            i = 0
            chart_data = []
            info_2 = info
            while i < len(info):
                m = info_2.find(r"[")
                n = info_2.find(r"]")
                i = i + n + 2
                chart_data.append(info_2[m+1:n])
                info_2 = info[i:]

            #Dates data
            chart_data_dates = chart_data[0].split(',')
            for i in range(0, len(chart_data_dates)):
                chart_data_dates[i] = re.sub(r'\"', '', chart_data_dates[i])
            latest_date.append(chart_data_dates[-1])

            #Getting Price Data
            chart_data_price = chart_data[1].split(',')
            chart_data_price = chart_data_price[1:]
            chart_data_price[0] = chart_data_price[0][8:]
            for i in range(0, len(chart_data_price)):
                chart_data_price[i] = re.sub(r'\"', '', chart_data_price[i])
                chart_data_price[i] = float(chart_data_price[i])

            latest_price.append(chart_data_price[-1])

            date_avg_price_dict = {chart_data_dates[i] :chart_data_price[i] for i in range(0, len(chart_data_price))}
            df_2 = pd.DataFrame({'Date': chart_data_dates, 'Price (Euros)': chart_data_price})
            
            #take the price data and puts it in a CSV
            df_2.to_csv('Cardmarket data\{}\{} {} price.csv'.format(self.set_name, list(df['Set Number'])[j], list(df['Name'])[j]))

            
#All above works on MP22! Testing on POTE

NameError: name 'POTE' is not defined

In [120]:
#Does not work - need to split starlight rares and add in the promo ultra rare.

POTE = TCG_set("https://yugipedia.com/wiki/Power_of_the_Elements")

ValueError: Length of values (106) does not match length of index (101)

In [ ]:
print(POTE.set_name, POTE.url, POTE.set_content, POTE)

In [116]:
#Test that it works, takes time to load lmao takes a while to get all the links.
MP22 = TCG_set("https://yugipedia.com/wiki/2022_Tin_of_the_Pharaoh%27s_Gods")
print(MP22.set_name, MP22.url, MP22.set_content)

2022 Tin of the Pharaoh's Gods https://yugipedia.com/wiki/2022_Tin_of_the_Pharaoh%27s_Gods      Set Number                       Name                 Rarity  \
0    MP22-EN001  Armed Dragon Thunder LV10             Ultra Rare   
1    MP22-EN002   Armed Dragon Thunder LV7  Prismatic Secret Rare   
2    MP22-EN003   Armed Dragon Thunder LV5                 Common   
3    MP22-EN004   Armed Dragon Thunder LV3                 Common   
4    MP22-EN005    Armed Dragon LV10 White                   Rare   
..          ...                        ...                    ...   
270  MP22-EN271      Link into the VRAINS!  Prismatic Secret Rare   
271  MP22-EN272         Soul Energy MAX!!!             Ultra Rare   
272  MP22-EN273        The Revived Sky God             Ultra Rare   
273  MP22-EN274      The Breaking Ruin God             Ultra Rare   
274  MP22-EN275           The True Sun God             Ultra Rare   

             Category    Print  \
0      Effect Monster  Reprint   
1      Effec

DDD Supersight King Zero Maxwell
Binary Blader
Sunavalon Daphne
Sunavalon Melias
Sunvine Cross Breed
Beetrooper Scout Buggy
Converging Wills Dragon
Stardust Synchron
Stardust Trail
Despian Comedy
Despian Tragedy
Aluber the Jester of Despia
Dramaturge of Despia
Albion the Shrouded Dragon
Clavkiys, the Magikey Skyblaster
Gunkan Suship Shari
Gunkan Suship Ikura
Chronomaly Magella Globe
Gizmek Inaba, the Hopping Hare of Hakuto
Gizmek Naganaki, the Sunrise Signaler
Gizmek Taniguku, the Immobile Intellect
Gizmek Arakami, the Hailbringer Hog
Carpiponica, Mystical Beast of the Forest
Glacier Aqua Madoor
Master's Diploman
Slower Swallow
Aeropixthree
Magikey Mechmusket - Batosbuster
Magikey Mechmortar - Garesglasser
Despian Quaeritis
Despian Proskenion
Magikey Beast - Ansyalabolas
Magikey Dragon - Andrabime
Allvain the Essence of Vanity
Stellar Wind Wolfrayet
Gaiarmor Dragonshell
Gunkan Suship Ikura-class Dreadnought
Chronomaly Vimana
Voloferniges, the Darkest Dragon Doomrider
Dragonlark Pairen


In [24]:
df = pd.read_csv('2022 Tin of the Pharoh\'s Gods (amended).csv')

In [26]:
df_common = df[df["Rarity"] == "Common"]
df_rare = df[df["Rarity"] == "Rare"]
df_super = df[df["Rarity"] == "Super Rare"]
df_ultra = df[df["Rarity"] == "Ultra Rare"]
df_secret = df[df["Rarity"] == "Prismatic Secret Rare"]

In [32]:
#what to do next?? I suppose get a way to get prices of this/ other recent sets and then get a way to automate that 
#and upload it to Kaggle

432